In [1]:
"""
spark.conf.set(
  "fs.azure.account.key.aqi-files.blob.core.windows.net",
  "ImXVWWw4HFH+HrL87Rd8oJuoCYI4XNz841tEbaKcaMxnAB2ByJiOwuqvjpv1uqIUUBWoLTJ8bYdHh4mCgFSnSw==")

dbutils.fs.mount(
  source = "wasbs://aqi-files@filestorageaqi.blob.core.windows.net",
  mount_point = "/mnt/AQI",
  extra_configs = {"fs.azure.account.key.filestorageaqi.blob.core.windows.net":"ImXVWWw4HFH+HrL87Rd8oJuoCYI4XNz841tEbaKcaMxnAB2ByJiOwuqvjpv1uqIUUBWoLTJ8bYdHh4mCgFSnSw=="})
"""

In [2]:
import pandas as pd

In [3]:
cityDay = spark.read.format('csv').options(header='true', inferSchema='true').load('/mnt/AQI/cityDayBlob.csv')
cityHour = spark.read.format('csv').options(header='true', inferSchema='true').load('/mnt/AQI/cityHourBlob.csv')


In [4]:
cityDayPd = cityDay.toPandas()
cityHourPd = cityHour.toPandas()

In [5]:
cityDayPd.interpolate(limit_direction="both",inplace=True)
cityHourPd.interpolate(limit_direction="both",inplace=True)

In [6]:
cityDayPd.loc[cityDayPd.AQI.between(0,51) , "AQI_Bucket" ] = "Good"
cityDayPd.loc[cityDayPd.AQI.between(51,101) , "AQI_Bucket" ] = "Satisfactory"
cityDayPd.loc[cityDayPd.AQI.between(101,201) , "AQI_Bucket" ] = "Moderate"
cityDayPd.loc[cityDayPd.AQI.between(201,301) , "AQI_Bucket" ] = "Poor"
cityDayPd.loc[cityDayPd.AQI.between(301,401) , "AQI_Bucket" ] = "Very Poor"
cityDayPd.loc[cityDayPd.AQI.between(401,3000) , "AQI_Bucket" ] = "Severe"

In [7]:
cityHourPd.loc[cityHourPd.AQI.between(0,51) , "AQI_Bucket" ] = "Good"
cityHourPd.loc[cityHourPd.AQI.between(51,101) , "AQI_Bucket" ] = "Satisfactory"
cityHourPd.loc[cityHourPd.AQI.between(101,201) , "AQI_Bucket" ] = "Moderate"
cityHourPd.loc[cityHourPd.AQI.between(201,301) , "AQI_Bucket" ] = "Poor"
cityHourPd.loc[cityHourPd.AQI.between(301,401) , "AQI_Bucket" ] = "Very Poor"
cityHourPd.loc[cityHourPd.AQI.between(401,3000) , "AQI_Bucket" ] = "Severe"

In [8]:
cityDayPd['Date'] =  pd.to_datetime(cityDayPd['Date'])
cityHourPd['Datetime'] =  pd.to_datetime(cityHourPd['Datetime'])


In [9]:
cityDayPd['Year'] = cityDayPd['Date'].dt.year
cityDayPd['Month'] = cityDayPd['Date'].dt.month
cityDayPd['Week_Day'] = cityDayPd['Date'].dt.day_name()

In [10]:
cityHourPd['Year'] = cityHourPd['Datetime'].dt.year
cityHourPd['Month'] = cityHourPd['Datetime'].dt.month
cityHourPd['Week_Day'] = cityHourPd['Datetime'].dt.day_name()

In [11]:
cityDayPd = spark.createDataFrame(cityDayPd)
cityHourPd = spark.createDataFrame(cityHourPd)

In [12]:
cityDayPd.coalesce(1).write.option("header", "true").format("com.databricks.spark.csv").save("/data/city_day_clean")

for f in dbutils.fs.ls("dbfs:/data/city_day_clean"):
    if f.path.endswith(('.csv')):
      csv_file_cityDay = f.path
      
cityHourPd.coalesce(1).write.option("header", "true").format("com.databricks.spark.csv").save("/data/city_hour_clean")

for f in dbutils.fs.ls("dbfs:/data/city_hour_clean"):
    if f.path.endswith(('.csv')):
      csv_file_cityHour = f.path



In [13]:

output_file_location1 = "/mnt/AQI/cityDay.csv"
output_file_location2 = "/mnt/AQI/cityHour.csv"

dbutils.fs.cp(csv_file_cityDay,output_file_location1)
dbutils.fs.cp(csv_file_cityHour,output_file_location2)

In [14]:
dbutils.fs.rm("/data",True)